In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [2]:
from collections import OrderedDict
from numpy import array
from math import sqrt
import sys
import os
import numpy
import urllib
import pyspark
from pyspark import SparkContext
from pyspark.mllib.feature import StandardScaler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import DenseVector
from pyspark.mllib.linalg import SparseVector
from collections import OrderedDict
from time import time

In [4]:
conf = (SparkConf()
         .setMaster("local[1]")
         .setAppName("kddcup.data.gz local[1]")
         .set("spark.executor.memory", "512M")
         .set("spark.cores.max", 4))
# sc.stop()
sc = SparkContext(conf = conf)

In [5]:
import urllib.request

f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz", "kddcup.data.gz")

In [6]:
kddcup_data = sc.textFile("file:///home/hduser/sandbox/kddcup.data.gz")

In [7]:
def parse_interaction(line):
    line_split = line.split(",")
    clean_line_split = [line_split[0]]+line_split[4:-1]
    return (line_split[-1], array([float(x) for x in clean_line_split]))

parsed_data = kddcup_data.map(parse_interaction)
pd_values = parsed_data.values().cache()

In [8]:
kdd_sample = pd_values.sample(False, .10, 123)
kdd_sample.count()

489191

In [9]:
standardizer = StandardScaler(True, True)
standardizer_model = standardizer.fit(kdd_sample)
data_for_cluster = standardizer_model.transform(kdd_sample)

In [10]:
k_values = range(10,31,10)
metrics = []

In [12]:
def error(point): 
    center = clusters.centers[clusters.predict(point)] 
    denseCenter = DenseVector(numpy.ndarray.tolist(center)) 
    return sqrt(sum([x**2 for x in (DenseVector(point.toArray()) - denseCenter)]))

In [13]:
#Fails because it cannot find file in other nodes. 
#What if i try using local[1]?
for k in k_values:
     clusters = KMeans.train(data_for_cluster, k, maxIterations=4, runs=5, initializationMode="random")
     WSSSE = data_for_cluster.map(lambda point: error(point)).reduce(lambda x, y: x + y)
     results = (k,WSSSE)
     metrics.append(results)

metrics

/opt/spark/python/pyspark/mllib/clustering.py:347: UserWarning: The param `runs` has no effect since Spark 2.0.0.
  warnings.warn("The param `runs` has no effect since Spark 2.0.0.")


[(10, 694538.8491664116), (20, 414102.2721093745), (30, 352680.3941882866)]

In [16]:
import time

t0 = time.time()

clusters = KMeans.train(data_for_cluster, 30, maxIterations=4, runs=5, initializationMode="random")
WSSSE = data_for_cluster.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print(30, WSSSE)

tt = time.time() - t0


print(tt)

/opt/spark/python/pyspark/mllib/clustering.py:347: UserWarning: The param `runs` has no effect since Spark 2.0.0.
  warnings.warn("The param `runs` has no effect since Spark 2.0.0.")


30 329911.20874727593
1722.3157720565796
